# Mixture 3D Subsequence DTW

This notebook implements an offline Mixture 3D Subsequence DTw approach.  Running this algorithm requires installing some other software, which is described below.  This notebook implements the `offline_processing()` and `online_processing()` functions, which will be imported and run in `02_RunExperiment.ipynb`.

Our main findings are:
- Because this algorithm performs DTW on a 3D cost tensor, it is computationally expensive.  This forced us to use highly downsampled features in order to achieve reasonable runtimes.
- With the downsampled features, we did not see any promising results.

## Offline Processing

In the offline processing stage, two things are computed and stored in the `cache/` folder:
- chroma features for the orchestra recording
- chroma features for the full mix recording

In [ ]:
import numpy as np
import pandas as pd
import import_ipynb
import system_utils
import align_tools
import sonify_tools
import os
import os.path
import subprocess
import librosa as lb
from shutil import which
from hmc_mir.align import dtw
import matplotlib.pyplot as plt
from numba import jit, njit, prange
import k3d

In [ ]:
def offline_processing(scenario_dir, cache_dir, hop_length=2048, downsample=10):
    '''
    Carries out the same offline processing steps as the simple offline DTW system.
    
    Args:
        scenario_dir: The scenario directory to process
        cache_dir: The location of the cache directory
        hop_length: The hop length in samples used when computing chroma features
        downsample: The downsampling factor used when computing chroma_sens features
    
    This function will store the computed chroma features and estimated alignment in the cache folder.
    '''
    # setup
    system_utils.verify_scenario_dir(scenario_dir)
    if os.path.exists(cache_dir):
        # print(f'{cache_dir} has already been processed.  Skipping.')
        pass
    else:
        # setup
        os.makedirs(cache_dir)
        
        o_file = f'{scenario_dir}/o.wav'
        y_o, sr = lb.core.load(o_file)
        F_o = lb.feature.chroma_cens(y=y_o, sr=sr, hop_length=hop_length)
        F_o = F_o[:,::downsample]

        po_file = f'{scenario_dir}/po.wav'
        y_po, sr = lb.core.load(po_file)
        F_po = lb.feature.chroma_cens(y=y_po, sr=sr, hop_length=hop_length)
        F_po = F_po[:,::downsample]

        np.save(f'{cache_dir}/po_cqt.npy', F_po / np.linalg.norm(F_po, axis=0))
        np.save(f'{cache_dir}/o_cqt.npy', F_o / np.linalg.norm(F_o, axis=0))

    return

In [ ]:
def verify_cache_dir(indir):
    '''
    Verifies that the specified cache directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/po_cqt.npy'), f'missing {indir}/po_cqt.npy'
    assert os.path.exists(f'{indir}/o_cqt.npy'), f'missing {indir}/o_cqt.npy'

## Online Processing

In the online processing stage, we do the following:
- We perform pairwise alignment between the P and PO recordings using standard subsequence DTw with chroma features.  This allows us to identify the matching portion of the PO recording.
- We then perform a 3D subsequence DTW alignment among P, PO-match, and the complete O recording using a mixture-based cost (dissimilarity between P+O and PO)

### Wrapper Implementation

In [ ]:
def online_processing(
    scenario_dir,
    out_dir,
    cache_dir,
    hop_length=2048,
    downsample=10,
    steps=np.array([[0,0,1],[1,1,1],[1,2,1],[2,1,1]]),
    weights=[0,1,1,2],
    match_buffer = 10
):
    '''
    Carries out `online' processing using the ISA algorithm.
    
    Args:
        scenario_dir: The scenario directory to process
        out_dir: The directory to put results, intermediate files, and logging info
        cache_dir: The cache directory
        hop_length: The hop length in samples used when computing cqt features
        downsample: The downsampling factor used when computing chroma_sens features

    This function will compute and save the predicted alignment in the output directory in a file hyp.npy
    '''
    
    # verify & setup
    system_utils.verify_scenario_dir(scenario_dir)
    verify_cache_dir(cache_dir)
    assert not os.path.exists(out_dir), f'Output directory {out_dir} already exists.'
    os.makedirs(out_dir)
    
    scenario_info = system_utils.get_scenario_info(scenario_dir+'/scenario.info')
    
    p_file = f'{scenario_dir}/p.wav'
    y, sr = lb.core.load(p_file)

    F_p = lb.feature.chroma_cens(y=y, sr=sr, hop_length=hop_length)[:,::downsample]
    F_p = F_p / np.linalg.norm(F_p, axis=0)
    F_po = np.load(f'{cache_dir}/po_cqt.npy') # full mix features
    F_o = np.load(f'{cache_dir}/o_cqt.npy') # orchestra
    
    F_p = np.nan_to_num(F_p)
    F_o = np.nan_to_num(F_o)
    F_po = np.nan_to_num(F_po)

    hop_sec = downsample * hop_length / sr

    C = 1 - (F_p.T @ F_po)
    D, B, path = dtw.dtw(C, np.array([[1,1],[1,2],[2,1]]),[1,1,2], True)
    F_po_match = F_po[:, max(path[1,0]-match_buffer, 0):min(path[1,-1]+match_buffer, F_po.shape[1])]

    best_cost, path = dtw3d(F_p, F_o, F_po_match, steps, weights)

    
    # infer piano-orchestra alignment
    wp_AC = np.vstack((path[0],path[1]))
    wp_AC[1, :] -= match_buffer
    wp_AC_sec = wp_AC*hop_sec
    wp_AC_sec[1,:] += scenario_info['oStart']
    np.save(f'{out_dir}/hyp.npy', wp_AC_sec)

    return

In [ ]:
def verify_hyp_dir(indir):
    '''
    Verifies that the specified scenario hypothesis directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/hyp.npy'), f'{indir} is missing the required files, please re run the online processing'

In [ ]:
@njit(parallel=True)
def calculate_cost_tensor(x, y, z):
    '''
    Calculates the cost tensor for the given piano, orchestra, and piano-orchestra features.
    
    Args:
        x: submix features
        y: submix features
        z: full mix features
    '''
    x = x.T
    y = y.T
    z = z.T
    cost_tensor = np.zeros((x.shape[0], y.shape[0], z.shape[0]))
    for i in prange(x.shape[0]):
        for j in prange(y.shape[0]):
            for k in prange(z.shape[0]):
                cost_tensor[i, j, k] = 1 - (np.dot(z[k,:], x[i,:] + y[j,:]) / (np.linalg.norm(z[k,:]) * np.linalg.norm(x[i,:] + y[j,:])))
    cost_tensor = np.swapaxes(cost_tensor, 1, 2)
    return cost_tensor

In [ ]:
@jit(nopython=True)
def backtrace_flexdtw(D, B, steps, rstart, cstart, tstart):
    '''
    Backtraces through the cumulative cost matrix D starting from a specified location.
    
    Arguments:
    D: cumulative cost matrix
    B: backtrace matrix
    steps: a numpy matrix specifying the allowable transitions.  It should be of
            dimension (L, 2), where each row specifies (row step, col step)
    rstart: the row index to start backtracking from
    cstart: the column index to start backtracking from
    
    Outputs
    path: a python list of (row, col) coordinates for the optimal path.
    '''
    pos = (rstart, cstart, tstart)
    path = []
    path.append(pos)
    while(pos[0] != 0 and pos[1] != 0 and pos[2] != 0):
        (row, col, tube) = pos
        stepidx = B[row, col, tube]
        (rstep, cstep, tstep) = steps[stepidx]
        pos = (row-rstep, col-cstep, tube-tstep)
        path.append(pos)
    
    return path

In [ ]:
@jit(nopython=True)
def find_best_endpoint(D, P, buffer):
    '''
    Determines the best location to begin backtracking from by comparing the average path cost
    per manhattan block.
    
    Inputs
    D: the cumulative cost matrix
    P: the matrix specifying the starting location of the alignment path
    buffer: specifies the length of a buffer region (in frames) to avoid short degenerate alignment paths
        near the corners of the pairwise cost matrix.  This can be thought of as the minimum length that
        needs to match in order to be considered a valid alignment path.
    
    Outputs
    best_cost: the best average path cost per manhattan block
    best_r: the row index of the best endpoint
    best_c: the column index of the best endpoint
    debug: debugging information for examining the average cost per manhattan block for each 
        of the candidate ending positions
    '''
    
    # consider the top corner 3 faces as candidates
    candidates = []
    for i in range(buffer, D.shape[0]):
        for j in range(buffer, D.shape[1]):
            candidates.append((i, j, D.shape[2]-1))

    for j in range(buffer, D.shape[1]):
        for k in range(buffer, D.shape[2]):
            candidates.append((D.shape[0]-1, j, k))

    for i in range(buffer, D.shape[0]):
        for k in range(buffer, D.shape[2]):
            candidates.append((i, D.shape[1]-1, k))
    
    best_cost = np.inf
    best_r, best_c, best_t = -1, -1, -1
    # debug = []
    debug = np.zeros_like(D)
    
    for i, (r,c,t) in enumerate(candidates):
                
        # get alignment start location
        rstart, cstart, tstart = P[r,c,t]
            
        # calculate average cost per manhattan block
        mdist = (r - rstart) + (c - cstart) + (t - tstart)# manhattan distance
        avg_cost_per_mb = D[r,c,t] / mdist
        
        # keep best
        if avg_cost_per_mb < best_cost:
            best_cost = avg_cost_per_mb
            best_r, best_c, best_t = r, c, t
            
        # debugging info
        # TODO: updated this for 3D-FlexDTW
        # if r == D.shape[0]-1:
        #     debug.append((c-D.shape[1]+1, avg_cost_per_mb, r, c))
        # else:
        #     debug.append((D.shape[0]-1-r, avg_cost_per_mb, r, c))
        debug[r,c,t] = avg_cost_per_mb
    
    return best_cost, best_r, best_c, best_t, debug

In [ ]:
@jit(nopython=True)
def dtw3d(x, y, z, steps, weights):
    C = calculate_cost_tensor(x, y, z)
    
    # Run FlexDTW on the cost tensor
    D = np.ones(C.shape) * np.inf
    B = np.zeros(C.shape, dtype=np.int8)

    # D[0, :, :] = np.inf
    # D[:, 0, :] = np.inf
    # D[:, :, 0] = np.inf
    # for row in range(1,C.shape[0]):
    #     for col in range(1, C.shape[1]):
    #         for tube in range (1, C.shape[2]):
    #             D[row, col, tube] = -np.inf
    # D[0,0,0] = C[0,0,0]
    D[0,:,:] = C[0,:,:]

    # DP
    for row in range(1,C.shape[0]):
        for col in range(1, C.shape[1]):
            for tube in range (1, C.shape[2]):
                mincost = np.inf
                minidx = -1
                bestrprev = -1
                bestcprev = -1
                besttprev = -1
                for stepidx, step in enumerate(steps):
                    (rstep, cstep, tstep) = step
                    prevrow = row - rstep
                    prevcol = col - cstep
                    prevtube = tube - tstep
                    if prevrow >= 0 and prevcol >= 0 and prevtube >= 0:
                        
                        pathcost = D[prevrow, prevcol, prevtube] + C[row, col, tube] * weights[stepidx]
                        
                        if pathcost < mincost:
                            mincost = pathcost
                            minidx = stepidx
                            bestrprev = prevrow
                            bestcprev = prevcol
                            besttprev = prevtube
                            
                D[row, col, tube] = D[bestrprev, bestcprev, besttprev] + C[row, col, tube] * weights[minidx]
                B[row, col, tube] = minidx

    best_cost = np.inf
    bestr = C.shape[0]-1
    bestc = -1
    bestt = -1
    for col in range(C.shape[1]):
        for tube in range(C.shape[2]):
            if D[bestr, col, tube] < best_cost:
                best_cost = D[bestr, col, tube]
                bestc = col
                bestt = tube

    path = backtrace_flexdtw(D, B, steps, bestr, bestc, bestt)
    path.reverse()
    path = np.array(path)

    return best_cost, path.T


# Example

Here is an example of how to call the offline and online processing functions on a scenario directory.

In [ ]:
# online_processing('scenarios/s1', 'experiments/mixtureDTW/s1', 'experiments/mixtureDTW/cache/rach2_mov1_O1_PO1')

In [ ]:
scenario_dir = 'scenarios/s12'
hop_length = 512
downsample = 10
#match_buffer = 10

In [ ]:
# extract O, PO features
o_file = f'{scenario_dir}/o.wav'
y_o, sr = lb.core.load(o_file)
F_o = lb.feature.chroma_cens(y=y_o, sr=sr, hop_length=hop_length)
F_o = F_o[:,::downsample]
# F_o = F_o / np.linalg.norm(F_o, axis=0)

po_file = f'{scenario_dir}/po.wav'
y_po, sr = lb.core.load(po_file)
F_po = lb.feature.chroma_cens(y=y_po, sr=sr, hop_length=hop_length)
F_po = F_po[:,::downsample]
# F_po = F_po / np.linalg.norm(F_po, axis=0)

In [ ]:
# extract P features
p_file = f'{scenario_dir}/p.wav'
y_p, sr = lb.core.load(p_file)
F_p = lb.feature.chroma_cens(y=y_p, sr=sr, hop_length=hop_length)
F_p = F_p[:,::downsample]
# F_p = F_p / np.linalg.norm(F_p, axis=0)

In [ ]:
# pre-select the matching region of PO
hop_sec = downsample * hop_length / sr
C = 1 - (F_p.T @ F_po)
D, B, path = dtw.dtw(C, np.array([[1,1],[1,2],[2,1]]),[1,1,2], True)

In [ ]:
po_match_start = path[1,0]
po_match_end = path[1,-1] + 1
F_po_match = F_po[:, po_match_start:po_match_end]

In [ ]:
F_p.shape[1], F_o.shape[1], F_po_match.shape[1], F_po.shape[1]

In [ ]:
steps=np.array([[0,0,1],[1,1,1],[1,2,1],[2,1,1]])
weights=[0.3,1.0,1.0,2.0]

In [ ]:
best_cost, path = dtw3d(F_p, F_o, F_po_match, steps, weights) 
# path[0] --> P
# path[1] --> PO
# path[2] --> O

In [ ]:
# P-O alignment
wp_AC = np.vstack((path[0],path[2]))
wp_AC_sec = wp_AC*hop_sec
plt.plot(wp_AC_sec[0,:], wp_AC_sec[1,:])
plt.xlabel('Piano (sec)')
plt.ylabel('Orchestra (sec)')
wp_AC_sec[1,0], wp_AC_sec[1,-1]

In [ ]:
# PO-O alignment
wp_BC = np.vstack((path[1],path[2]))
wp_BC_sec = wp_BC*hop_sec
plt.plot(wp_BC_sec[0,:], wp_BC_sec[1,:])
plt.xlabel('Full Mix (sec)')
plt.ylabel('Orchestra (sec)')

In [ ]:
# P-PO alignment
wp_AB = np.vstack((path[0],path[1]))
wp_AB_sec = wp_AB*hop_sec
plt.plot(wp_AB_sec[0,:], wp_AB_sec[1,:])
plt.xlabel('Piano (sec)')
plt.ylabel('Full Mix (sec)')

In [ ]:
wp_AC_sec[:,-5:]

In [ ]:
N = 1000
F_random = np.random.rand(12,N)

In [ ]:
%%time
best_cost, path = dtw3d(F_random, F_random, F_random, steps, weights)